# Order and Download Data with the Capella API
This script is a template for ordering and downloading an image from the archive using the Capella API. It walks through an example using a scene from the `capella-open-data` collection. 

The cells that require the user to input parameters prior to running are marked with `USER INPUT REQUIRED` in the header. Searching for `USER` in the notebook will also identify where the user must enter inputs.

* **Author:** [Hayley Pippin](mailto:hayley.pippin@capellaspace.com)
* **Last updated:** August 9, 2024
* **Required input(s):**
    * `credentials.json`: JSON containing the user's Capella Console credentials.
* **Output(s):**
    * Links to all STAC assests associated with the ordered image

## Setup
Unless otherwise noted, these cells must be run every time you use this script. 

### `credentials.json`
Your username and password must be saved in a `.json` file named `credentials.json` and formatted as follows:
```
{"username": "yourusername","password": "xxxxxxxxx"}
```

### Install packages
The following cell **only needs to be run once** if packages are not already installed. Uncomment any of the following lines to install the necessary packages.

In [ ]:
#!pip install requests
#!pip install json

### Import packages and define helper functions

In [ ]:
import requests
import json

# Helper function to printformatted JSON using the json module
def p(data):
    print(json.dumps(data, indent=2))

### Authentication (INITIAL USER INPUT REQUIRED)
This cell needs to be run hourly to re-authenticate with the Capella system.

In [ ]:
# Load username and password
with open('WRITE PATH TO CREDENTIALS FILE HERE') as f: # USER: Input path to credentials.json file.
    data = json.load(f)
    username = data['username']
    password = data['password']

# Get a valid token from the auth service
r = requests.post("https://api.capellaspace.com/token", 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))
access_token = r.json()["accessToken"]
# p(accessToken)

# GET user ID and org ID
headers = {'Authorization':'Bearer ' + access_token}
r = requests.get("https://api.capellaspace.com/user", headers=headers)
user_id = r.json()["id"]
org_id = r.json()["organizationId"]
#p(r.json())

# Print user ID, org ID, and current environment
print('User email: ', r.json()['email'], '\nOrganization: ', r.json()['organization']['name'], '\nEnvironment: ', r.json()['apiEnvironmentRole'])

## Order Data

### Using a Collect ID (USER INPUT REQUIRED)

In [ ]:
order_body = {
    "items": [
        {
            "collectionId": "capella-open-data", # USER: Input image collection ID here.
            "granuleId": "CAPELLA_C02_SP_GEO_HH_20210106050228_20210106050252" # USER: Input image STAC ID here.
        }
    ]
}

In [ ]:
r = requests.post("https://api.capellaspace.com/orders/", json = order_body, headers = headers)

# print("Order ID:", r.json()["orderId"]) # Print the order ID
p(r.json()) # Print the entire response body

## Download Data

### Define the order ID (USER INPUT REQUIRED)

In [ ]:
# order_id = "" # USER: Uncomment this line to input pre-existing order ID.
order_id = r.json()["orderId"] # USER: Uncomment this line to use the order ID generated in section above.

### Generate download links

In [ ]:
r = requests.get(f"https://api.capellaspace.com/orders/{order_id}/download", headers = headers)
p(r.json())